# Streaming

It's time to get hands-on with LSL. In this part, we'll be creating an LSL stream and capturing it with a server.

## Stream Outlet

Let's create a stream outlet that can be used to push single-channel data to the LSL network. The outlet needs to be constructed with information detailing:

- The name of the stream
- The type of the stream
- The number of channels the stream uses
- The sampling rate the stream is updated at
- The type of value the stream outputs
- A (should-be) unique identifier for the stream

In [ ]:
from pylsl import StreamInfo, StreamOutlet

def create_outlet():
    name = 'workshop_outlet'                                                                # The name of the outlet
    type = 'single_stream'                                                                  # Stream type. This can be anything, but should probably be descriptive of what data is going through (e.g. EDA, EEG, Markers)
    n_of_channels = 1                                                                       # Single-channel data
    samping_rate = 1                                                                        # 1Hz = 1 sample per second
    value_type = 'float32'                                                                  # The type that all stream values should be.
    outlet_id = 'workshop_outlet_1234'                                                      # An unique identifier used to resolve streams when pulling data from the LSL network

    stream = StreamInfo(name, type, n_of_channels, samping_rate, value_type, outlet_id)     # Establish the details of the stream
    outlet = StreamOutlet(stream)                                                           # Create the outlet

    return outlet

Next, let's define a function that generates random floating point numbers with max. 2 decimals and pushes them through the outlet. Note that since outlets can be used to push samples of various numbers of channels, we need to wrap our sample in an array `[]` when sending it, even if we defined the stream to be one channel earlier.



In [ ]:
import time
import random

def output_random_sample(target_outlet):
    random_sample = round(random.random(), 2)                                               # Generate a random floating point value between 0 and 1 and round it to 2 decimals.
    print('Outputting random sample:', random_sample)                                       # Print out the sample we just generated

    target_outlet.push_sample([random_sample])                                              # Push the sample through the outlet

Finally, let's try to run the function. You should see a new printout every second. 

Note three things about the code below:
- To keep the notebook clean, we are using the `clear_output` function from IPython to remove previous outputs.
- We defined our sampling rate to be 1Hz, so we're using Python's `time.sleep` to artificially delay each loop execution by 1 second
- As it has been hard-coded to run indefinitely, you can stop the output by pressing the stop button next to the cell.

In [ ]:
from IPython.display import clear_output

outlet = create_outlet()

while True:
    clear_output()                                                                          # Clear the output below the cell so that only one value is displayed at atime

    output_random_sample(outlet)
    time.sleep(1)                                                                           # Wait for 1 second until the next loop

## Stream Inlet

Now we know how to push data to the LSL network, but how do we make use of it? 

As mentioned in part 1, we now need to create an inlet. To create an inlet, we need to resolve available streams in the network. 

> You could resolve all streams at once if you wanted to, but here we'll filter out everything that doesn't possess our unique stream identifier. Take a look at [the liblsl documentation for the `stream_info` class](https://labstreaminglayer.readthedocs.io/projects/liblsl/ref/streaminfo.html?highlight=streaminfo) to see the parameter names that can be used to filter streams (here, `source_id` refers to the value we gave our `outlet_id` variable above).

In [ ]:
from pylsl import StreamInlet, resolve_stream

def create_inlet():
    stream = resolve_stream('source_id', 'workshop_outlet_1234')[0]                     # Find streams with our desired ID and grab the first one (the function returns an array)
    inlet = StreamInlet(stream)                                                         # Establish an inlet that data can be pulled through
    return inlet

Next, let's write a function that pulls a sample from an inlet. Notable here is that when pulling the sample we can also access its timestamp. The ease of this (no configuration required on our part) is one of the core features that make LSL such an attractive solution for research.

The `pull_sample` function returns a tuple, which we then destructure (unpack) to two variables: `samples`, which is an array containing our sample, and `ts`, which is the timestamp for when the sample was pushed through the outlet.

In [ ]:
def pull_random_sample(source_inlet):
    samples, ts = source_inlet.pull_sample()
    print('Received sample:', samples[0], 'with timestamp', ts)

Now, let's demonstrate how we can combine our outlet and inlet and run them in parallel to send and capture data in real time. 

In a realistic environment, you would probably have two separate scripts: one for gathering samples and pushing them through an outlet, and another for pulling samples from the LSL network and logging them. Due to the limitations of the notebook format, we cannot run two cells at once. That's why in the script below, we use `Thread` to run our random sampler & outlet functionality asynchronously in a separate CPU thread from the one running our inlet.

This time, the script runs for 20 seconds.

In [ ]:
from threading import Thread
from IPython.display import clear_output

clear_output()                                                   

# Demo: Run the sampler & outlet in a separate CPU thread to circumvent Jupyter limitations
# You probably don't need/want to do this in any real-world application (not threading, just running the outlet & inlet in a single script - because why use LSL in that case?)

def run_sampling():                              
    outlet = create_outlet()
    run_time = 22                                                   # Samples will be generated for 22 seconds

    while run_time > 0:
        output_random_sample(outlet)
        time.sleep(1)                                               # Artifically sleep for 1s between samples (1Hz sampling rate)
        run_time -= 1                                               # n-1 seconds left for sampling
    
    print('Sample output complete!')

sampler = Thread(target = run_sampling)                             # Assign our outlet to a separate CPU thread
sampler.start()

time.sleep(2)                                                       # Wait a couple of seconds to make sure the outlet is established

# Pull samples from the network

inlet = create_inlet()

pull_time = 20                                                      # Receive samples for 20 seconds (22 - 2)

while pull_time > 0:
    pull_random_sample(inlet)
    pull_time -= 1                                                  # the run_time variable has separate instances for this main thread and the sampling thread so we have to deduct this here as well

inlet.close_stream()                                                # Close the inlet! This allows our program to finish. Otherwise, LSL will hang indefinitely waiting to reconnect. This is a great safety feature but for the demo a bit annoying...

You might have noticed a couple of things above. First of all, the first sample that was pushed to the outlet was not captured by the inlet. This is because the inlet takes a bit to be initialized, making it recommended that you start your logger script slightly ahead of your sampler script. In the example, we could change this by e.g. adding a `sleep` of 1-2 seconds at the beginning of `run_sampling`.

Second, you might have noticed that while the received samples round out to the ones we push to the outlet, they are not exact. This is a floating point error that is inherent to Python's implementation of decimal values (see [this documentation page](https://docs.python.org/3/tutorial/floatingpoint.html) for more information). If you recall, our stream's type was defined as `float32`. If, somehow, the values we output to the outlet should be exactly two decimals long, we could **a)** multiply the rounded values by 10 and convert them to integers, **b)** convert them to [integer ratios](https://docs.python.org/3/library/stdtypes.html#float.as_integer_ratio) before pushing them to the network or **c)** render them as strings before pushing them to the network. 

**HOWEVER**, you probably will be working with devices that output floats with *n* decimals anyway, and the better practice is to **log everything raw** and do the processing during data analysis.

## Marker Streams

When working with continuous data, you usually want to be able to connect changes in the information to events in the real world. For example, if collecting the electrodermal activity of participants in order to measure activation when interacting with an application, you probably want to know when things within the application are updated (e.g. view change, key press, activity begins/ends). Similarly, in more controlled experiments, you want to know when a stimulus begins and ends in order to be able to segment the continuous data for analysis. Even if using something like a room temperature sensor, you might be interested in events like changing the air conditioning, turning on a certain electronic device or when the sensor is restarted.

Data points signifying discrete events are called **markers**. In LSL terms, a marker stream is no different from a regular stream - the difference is your implementation: markers are tied to activities and sent irregularly. You can use integers as markers, but a string (text) is usually more helpful, especially if you're sending trial configuration information.

Let's edit our code from earlier to set up a string marker outlet.

In [ ]:
def create_marker_outlet():
    name = 'workshop_marker_outlet'                                                         # The name of the outlet
    type = 'single_stream'                                                                  # Stream type. This can be anything, but should probably be descriptive of what data is going through (e.g. EDA, EEG, Markers)
    n_of_channels = 1                                                                       # Single-channel data
    sampling_rate = 0                                                                       # 0Hz = 0 samples / second -> Indicates irregular sampling rate
    value_type = 'string'                                                                   # The type that all stream values should be. This time we're sending strings.
    outlet_id = 'workshop_outlet_5678'                                                      # An unique identifier used to resolve streams when pulling data from the LSL network

    stream = StreamInfo(name, type, n_of_channels, sampling_rate, value_type, outlet_id)    # Establish the details of the stream
    outlet = StreamOutlet(stream)                                                           # Create the outlet

    return outlet

def output_marker(marker_text, target_outlet):                                              # Take the marker as a parameter
    print('Outputting random sample:', marker_text)                                         # Print out the marker

    target_outlet.push_sample([marker_text])                                                # Push the marker through the outlet

Besides changing the name and ID of the outlet, note that we changed the `value_type` to string, since we're sending string markers. We've also changed `sampling_rate` to 0, which indicates that samples (markers) are pushed to the outlet irregularly. In the `output_marker` function, we've now parameterized the marker contents.

Let's demonstrate this by sending different markers for different random values. When the value is greater or equal to 0.5, a marker with the text `right` is pushed. Likewise, `left` is pushed when the value is below 0.5.

In [ ]:
from IPython.display import clear_output

outlet = create_marker_outlet()

while True:
    clear_output()                                                                                      # Clear the output below the cell so that only one value is displayed at atime
    if random.random() >= 0.5:                                                                          
        output_marker('right', outlet)                                                                   # Push 'right' to the outlet
        time.sleep(1)                                                                                   # Wait for 1 second until the next loop
    else:
        output_marker('left', outlet)                                                                   # Push 'left' to the outlet
        time.sleep(1)                                                                                   # Wait for 1 second until the next loop


## Resolving Markers

Because LSL does not inherently differentiate between discrete (marker) and continuous (data) streams, data from marker streams is pulled in exactly the same way as with data streams. All the resolver (`resolve_streams`) needs to know is the parameter we use to identify each stream. In our case this was the ID of the stream.

Let's change our code from earlier to create a marker inlet and a function to pull markers through the inlet.

In [ ]:
def create_marker_inlet():
    stream = resolve_stream('source_id', 'workshop_outlet_5678')[0]                     # Get our marker stream
    inlet = StreamInlet(stream)                                                         # Establish an inlet that data can be pulled through
    return inlet


def pull_marker(source_inlet):                                                         
    samples, ts = source_inlet.pull_sample()
    print('Received marker:', samples[0], 'with timestamp', ts)

Note that the `pull_marker` function is functionally not any different from our `pull_random_sample` function from earlier. Since both our continuous data stream and marker stream are single-channel this time, we could just use one or the other to handle pulling samples from both.

## Exercises

The workshop exercises can be found in [the excercises folder](./exercises/). 

### A. Receiving markers

The directory contains three files. 
- `marker_gen.py` contains code, that, similar to our example above, pushes string markers to an LSL stream outlet based on a random number. 
- `logger.py` contains a loop that attempts to pull samples from an inlet. 

Your task is to update the script in `logger.py` to receive and print string markers sent from the outlet in `marker_gen.py`.

You can test your code by opening two terminal windows side-by-side. Activating the workshop's conda environment and navigating to the exercise directory in each, and then executing the scripts with Python:

```bash
$ python scriptnamehere.py
```

**Do not change the code in `marker_gen.py`!**

### B. Multiple streams

The directory contains three files. `logger.py` runs a loop that attempts to fetch markers and samples from two inlets with the IDs `b-marker-stream` and `b-sample-stream`, and then prints any received samples to the terminal.

`marker_gen.py`, `sample_gen.py` only contain the imports necessary to complete the exercise. Your task is to write a script in both files so that `marker_gen` pushes a marker to a stream with the ID `b-marker stream` when a random float is greater than 0.85, while `sample_gen.py` pushes random floating point numbers between 0 and 1000 at a 10Hz sample rate. You can use the example code above as a starting point.

Because the sample rates of the two streams are not the same, we have to pull data in chunks! Look at `logger.py` for how it's been implemented. You don't need to push chunks to the stream in this exercise.

To make it so that you don't need to run three terminal windows manually, helper scripts are included that will run all the tree programs for you. 

If you're on linux, run

```bash
$ ./run.sh
```

If you're on MacOS, change the file paths in ```run_osx.sh``` to match the exercise directory on your computer. Then, run

```bash
$ ./run_osx.sh
```

> If the run scripts don't work, you can try to give them execution rights with
>
> ```bash
> $ chmod u+x scriptname.sh
> ```
>
> If you still are unable to run, you can open three terminals and execute the scripts manually as in exercise A.

### C. Multichannel stream

The directory contains the example random float sampling code from the beginning of part 2, split into two files. `sample_gen.py` generates samples and pushes them to an outlet, while `logger.py` reads samples from a stream and prints them out.

Your task is to adapt the code so that instead of a single float value at a time, an 8-item array of float values goes through. Make sure to update both the inlet and the outlet!

You can use `run.sh` or `run_osx.sh` as in the other exercises to test your code.